# Polygon IO

>[Polygon](https://polygon.io/) The Polygon.io Stocks API provides REST endpoints that let you query the latest market data from all US stock exchanges.

This notebook uses tools to get stock market data like the latest quote and news for a ticker from Polygon.

In [1]:
import getpass
import os

os.environ["POLYGON_API_KEY"] = getpass.getpass()

 ········


In [2]:
from langchain_community.tools.polygon.aggregates import PolygonAggregates
from langchain_community.tools.polygon.financials import PolygonFinancials
from langchain_community.tools.polygon.last_quote import PolygonLastQuote
from langchain_community.tools.polygon.ticker_news import PolygonTickerNews
from langchain_community.utilities.polygon import PolygonAPIWrapper

In [3]:
api_wrapper = PolygonAPIWrapper()
ticker = "AAPL"

### Get latest quote for ticker

In [4]:
# Get the last quote for ticker
last_quote_tool = PolygonLastQuote(api_wrapper=api_wrapper)
last_quote = last_quote_tool.run(ticker)
print(f"Tool output: {last_quote}")

Tool output: {"P": 170.5, "S": 2, "T": "AAPL", "X": 11, "i": [604], "p": 170.48, "q": 106666224, "s": 1, "t": 1709945992614283138, "x": 12, "y": 1709945992614268948, "z": 3}


In [5]:
import json

# Convert the last quote response to JSON
last_quote = last_quote_tool.run(ticker)
last_quote_json = json.loads(last_quote)

In [6]:
# Print the latest price for ticker
latest_price = last_quote_json["p"]
print(f"Latest price for {ticker} is ${latest_price}")

Latest price for AAPL is $170.48


### Get aggregates (historical prices) for ticker

In [20]:
from langchain_community.tools.polygon.aggregates import PolygonAggregatesSchema

# Define param
params = PolygonAggregatesSchema(
    ticker=ticker,
    timespan="day",
    timespan_multiplier=1,
    from_date="2024-03-01",
    to_date="2024-03-08",
)
# Get aggregates for ticker
aggregates_tool = PolygonAggregates(api_wrapper=api_wrapper)
aggregates = aggregates_tool.run(tool_input=params.dict())
aggregates_json = json.loads(aggregates)

In [25]:
print(f"Total aggregates: {len(aggregates_json)}")
print(f"Aggregates: {aggregates_json}")

Total aggregates: 6
Aggregates: [{'v': 73450582.0, 'vw': 179.0322, 'o': 179.55, 'c': 179.66, 'h': 180.53, 'l': 177.38, 't': 1709269200000, 'n': 911077}, {'v': 81505451.0, 'vw': 174.8938, 'o': 176.15, 'c': 175.1, 'h': 176.9, 'l': 173.79, 't': 1709528400000, 'n': 1167166}, {'v': 94702355.0, 'vw': 170.3234, 'o': 170.76, 'c': 170.12, 'h': 172.04, 'l': 169.62, 't': 1709614800000, 'n': 1108820}, {'v': 68568907.0, 'vw': 169.5506, 'o': 171.06, 'c': 169.12, 'h': 171.24, 'l': 168.68, 't': 1709701200000, 'n': 896297}, {'v': 71763761.0, 'vw': 169.3619, 'o': 169.15, 'c': 169, 'h': 170.73, 'l': 168.49, 't': 1709787600000, 'n': 825405}, {'v': 76267041.0, 'vw': 171.5322, 'o': 169, 'c': 170.73, 'h': 173.7, 'l': 168.94, 't': 1709874000000, 'n': 925213}]


### Get latest news for ticker

In [ ]:
ticker_news_tool = PolygonTickerNews(api_wrapper=api_wrapper)
ticker_news = ticker_news_tool.run(ticker)

In [9]:
# Convert the news response to JSON array
ticker_news_json = json.loads(ticker_news)
print(f"Total news items: {len(ticker_news_json)}")

Total news items: 10


In [10]:
# Inspect the first news item
news_item = ticker_news_json[0]
print(f"Title: {news_item['title']}")
print(f"Description: {news_item['description']}")
print(f"Publisher: {news_item['publisher']['name']}")
print(f"URL: {news_item['article_url']}")

Title: An AI surprise could fuel a 20% rally for the S&P 500 in 2024, says UBS
Description: If Gen AI causes a big productivity boost, stocks could see an unexpected rally this year, say UBS strategists.
Publisher: MarketWatch
URL: https://www.marketwatch.com/story/an-ai-surprise-could-fuel-a-20-rally-for-the-s-p-500-in-2024-says-ubs-1044d716


### Get financials for ticker

In [28]:
financials_tool = PolygonFinancials(api_wrapper=api_wrapper)
financials = financials_tool.run(ticker)

In [30]:
# Convert the financials response to JSON
financials_json = json.loads(financials)
print(f"Total reporting periods: {len(financials_json)}")

Total reporting periods: 10


In [31]:
# Print the latest reporting period's financials metadata
financial_data = financials_json[0]
print(f"Company name: {financial_data['company_name']}")
print(f"CIK: {financial_data['cik']}")
print(f"Fiscal period: {financial_data['fiscal_period']}")
print(f"End date: {financial_data['end_date']}")
print(f"Start date: {financial_data['start_date']}")

Company name: APPLE INC
CIK: 0000320193
Fiscal period: TTM
End date: 2023-12-30
Start date: 2022-12-31


In [32]:
# Print the latest reporting period's income statement
print(f"Income statement: {financial_data['financials']['income_statement']}")

Income statement: {'diluted_earnings_per_share': {'value': 6.42, 'unit': 'USD / shares', 'label': 'Diluted Earnings Per Share', 'order': 4300}, 'costs_and_expenses': {'value': 267270000000.0, 'unit': 'USD', 'label': 'Costs And Expenses', 'order': 600}, 'net_income_loss_attributable_to_noncontrolling_interest': {'value': 0, 'unit': 'USD', 'label': 'Net Income/Loss Attributable To Noncontrolling Interest', 'order': 3300}, 'net_income_loss_attributable_to_parent': {'value': 100913000000.0, 'unit': 'USD', 'label': 'Net Income/Loss Attributable To Parent', 'order': 3500}, 'income_tax_expense_benefit': {'value': 17523000000.0, 'unit': 'USD', 'label': 'Income Tax Expense/Benefit', 'order': 2200}, 'income_loss_from_continuing_operations_before_tax': {'value': 118436000000.0, 'unit': 'USD', 'label': 'Income/Loss From Continuing Operations Before Tax', 'order': 1500}, 'operating_expenses': {'value': 55013000000.0, 'unit': 'USD', 'label': 'Operating Expenses', 'order': 1000}, 'benefits_costs_expe

In [34]:
# Print the latest reporting period's balance sheet
print(f"Balance sheet: {financial_data['financials']['balance_sheet']}")

Balance sheet: {'equity_attributable_to_noncontrolling_interest': {'value': 0, 'unit': 'USD', 'label': 'Equity Attributable To Noncontrolling Interest', 'order': 1500}, 'other_noncurrent_liabilities': {'value': 39441000000.0, 'unit': 'USD', 'label': 'Other Non-current Liabilities', 'order': 820}, 'equity': {'value': 74100000000.0, 'unit': 'USD', 'label': 'Equity', 'order': 1400}, 'liabilities': {'value': 279414000000.0, 'unit': 'USD', 'label': 'Liabilities', 'order': 600}, 'noncurrent_assets': {'value': 209822000000.0, 'unit': 'USD', 'label': 'Noncurrent Assets', 'order': 300}, 'equity_attributable_to_parent': {'value': 74100000000.0, 'unit': 'USD', 'label': 'Equity Attributable To Parent', 'order': 1600}, 'liabilities_and_equity': {'value': 353514000000.0, 'unit': 'USD', 'label': 'Liabilities And Equity', 'order': 1900}, 'other_current_liabilities': {'value': 75827000000.0, 'unit': 'USD', 'label': 'Other Current Liabilities', 'order': 740}, 'inventory': {'value': 6511000000.0, 'unit':

In [33]:
# Print the latest reporting period's cash flow statement
print(f"Cash flow statement: {financial_data['financials']['cash_flow_statement']}")

Cash flow statement: {'net_cash_flow_continuing': {'value': 20000000000.0, 'unit': 'USD', 'label': 'Net Cash Flow, Continuing', 'order': 1200}, 'net_cash_flow_from_investing_activities_continuing': {'value': 7077000000.0, 'unit': 'USD', 'label': 'Net Cash Flow From Investing Activities, Continuing', 'order': 500}, 'net_cash_flow_from_investing_activities': {'value': 7077000000.0, 'unit': 'USD', 'label': 'Net Cash Flow From Investing Activities', 'order': 400}, 'net_cash_flow_from_financing_activities_continuing': {'value': -103510000000.0, 'unit': 'USD', 'label': 'Net Cash Flow From Financing Activities, Continuing', 'order': 800}, 'net_cash_flow_from_operating_activities': {'value': 116433000000.0, 'unit': 'USD', 'label': 'Net Cash Flow From Operating Activities', 'order': 100}, 'net_cash_flow_from_financing_activities': {'value': -103510000000.0, 'unit': 'USD', 'label': 'Net Cash Flow From Financing Activities', 'order': 700}, 'net_cash_flow_from_operating_activities_continuing': {'v